In [10]:
# Write your imports here
import cv2
import numpy as np
import os

opencv байхгүй бол суулгах OPEN COMPUTER VISION library

In [11]:
#pip install opencv-python

# Working with Images Lab
## Information retrieval, preprocessing, and feature extraction

In this lab, you'll work with images of felines (cats), which have been classified according to their taxonomy. Each subfolder contains images of a particular species. The dataset is located [here](https://www.kaggle.com/datasets/datahmifitb/felis-taxonomy-image-classification) but it's also provided to you in the `data/` folder.

### Problem 1. Some exploration (1 point)
How many types of cats are there? How many images do we have of each? What is a typical image size? Are there any outliers in size?

In [12]:
path = 'data'

cat_types = {}  
image_sizes = []  

with os.scandir(path) as entries:
    for entry in entries:
        if entry.is_dir():
            folder_path = entry.path 
            files = [f for f in os.listdir(folder_path) if f.lower().endswith(('jpg', 'jpeg', 'png'))]  
            
            cat_types[entry.name] = len(files)  
            
            for file in files:
                img_path = os.path.join(folder_path, file)
                img = cv2.imread(img_path)
                if img is not None:
                    height, width, _ = img.shape
                    image_sizes.append(height * width) 
                else:
                    print(f"Error reading {img_path}")

print("Cat Types and Image Counts:")
for cat, count in cat_types.items():
    print(f"{cat}: {count} images")

if image_sizes:
    mean_size = np.mean(image_sizes)
    median_size = np.median(image_sizes)
    std_dev = np.std(image_sizes)

    outliers = [size for size in image_sizes if size > mean_size + 2 * std_dev]

    print(f"\nTypical Image Size (Mean): {mean_size:.2f} pixels")
    print(f"Median Image Size: {median_size:.2f} pixels")
    print(f"Number of Outlier Images: {len(outliers)}")
else:
    print("\nNo images found.")

Cat Types and Image Counts:
african-wildcat: 83 images
blackfoot-cat: 74 images
chinese-mountain-cat: 36 images
domestic-cat: 58 images
european-wildcat: 53 images
jungle-cat: 80 images
sand-cat: 67 images

Typical Image Size (Mean): 251861.33 pixels
Median Image Size: 50350.00 pixels
Number of Outlier Images: 10


### Problem 2. Duplicat(e)s (1 point)
Find a way to filter out (remove) identical images. I would recommnend using file hashes, but there are many approaches. Keep in mind that during file saving, recompression, etc., a lot of artifacts can change the file content (bytes), but not visually.

In [13]:
import hashlib
from imagehash import phash
from PIL import Image

md5_hashes = set()
phash_dict = {}

# Function to calculate MD5 hash of a file
def get_md5(file_path):
    hasher = hashlib.md5()
    with open(file_path, 'rb') as f:
        buf = f.read()
        hasher.update(buf)
    return hasher.hexdigest()

# Function to calculate perceptual hash (pHash)
def get_phash(image_path):
    try:
        img = Image.open(image_path)
        return phash(img)  # Returns a unique hash based on image content
    except Exception as e:
        print(f"Error hashing {image_path}: {e}")
        return None

# Iterate over all folders
for root, _, files in os.walk(path):
    for file in files:
        if file.lower().endswith(('jpg', 'jpeg', 'png')):
            img_path = os.path.join(root, file)

            # Check for exact duplicates using MD5
            md5 = get_md5(img_path)
            if md5 in md5_hashes:
                print(f"Removing exact duplicate: {img_path}")
                os.remove(img_path)
                continue  # Skip to the next file
            md5_hashes.add(md5)

            # Check for near duplicates using pHash
            img_phash = get_phash(img_path)
            if img_phash:
                # Compare with existing hashes (Hamming distance < 5 means similar)
                if any(img_phash - existing_hash < 5 for existing_hash in phash_dict.values()):
                    print(f"Removing near-duplicate: {img_path}")
                    os.remove(img_path)
                    continue
                phash_dict[img_path] = img_phash

print("✅ Duplicate removal complete.")

✅ Duplicate removal complete.


### Problem 3. Loading a model (2 points)
Find a suitable, trained convolutional neural network classifier. I recommend `ResNet50` as it's small enough to run well on any machine and powerful enough to make reasonable predictions. Most ready-made classifiers have been trained for 1000 classes.

You'll need to install libraries and possibly tinker with configurations for this task. When you're done, display the total number of layers and the total number of parameters. For ResNet50, you should expect around 50 layers and 25M parameters.

In [14]:
import torch
import torchvision.models as models

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

model = models.resnet50(pretrained=True)
model = model.to(device)

Using device: cuda


### Problem 4. Prepare the images (1 point)
You'll need to prepare the images for passing to the model. To do so, they have to be resized to the same dimensions. Most available models have a specific requirement for sizes. You may need to do additional preprocessing, depending on the model requirements. These requirements should be easily available in the model documentation.

In [15]:
target_size = (224, 224)

prepared_data = []

with os.scandir(path) as entries:
    for entry in entries:
        if entry.is_dir():
            label = entry.name  # Subfolder name is the label
            folder_path = entry.path 
            files = [f for f in os.listdir(folder_path) if f.lower().endswith(('jpg', 'jpeg', 'png'))]  
            
            cat_types[label] = len(files)  
            
            for file in files:
                img_path = os.path.join(folder_path, file)
                img = cv2.imread(img_path)
                if img is not None:
                    # Convert from BGR to RGB
                    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
                    # Resize the image to the target size
                    img = cv2.resize(img, target_size)
                    height, width, _ = img.shape
                    image_sizes.append(height * width)
                    # Append the processed image along with its label
                    prepared_data.append((img, label))
                else:
                    print(f"Error reading {img_path}")

print("Cat Types and Image Counts:")
for cat, count in cat_types.items():
    print(f"{cat}: {count} images")

if image_sizes:
    mean_size = np.mean(image_sizes)
    median_size = np.median(image_sizes)
    std_dev = np.std(image_sizes)

    outliers = [size for size in image_sizes if size > mean_size + 2 * std_dev]

    print(f"\nTypical Image Size (Mean): {mean_size:.2f} pixels")
    print(f"Median Image Size: {median_size:.2f} pixels")
    print(f"Number of Outlier Images: {len(outliers)}")
else:
    print("\nNo images found.")

print(f"\nPrepared {len(prepared_data)} images for training.")

Cat Types and Image Counts:
african-wildcat: 83 images
blackfoot-cat: 74 images
chinese-mountain-cat: 36 images
domestic-cat: 58 images
european-wildcat: 53 images
jungle-cat: 80 images
sand-cat: 67 images

Typical Image Size (Mean): 151018.66 pixels
Median Image Size: 50176.00 pixels
Number of Outlier Images: 11

Prepared 451 images for training.


### Problem 5. Load the images efficiently (1 point)
Now that you've seen how to prepare the images for passing to the model... find a way to do it efficiently. Instead of loading the entire dataset in the RAM, read the images in batches (e.g. 4 images at a time). The goal is to read these, preprocess them, maybe save the preprocessed results in RAM.

If you've already done this in one of the previous problems, just skip this one. You'll get your point for it.

\* Even better, save the preprocessed image arrays (they will not be valid .jpg file) as separate files, so you can load them "lazily" in the following steps. This is a very common optimization to work with large datasets.

### Problem 6. Predictions (1 point)
Finally, you're ready to get into the meat of the problem. Obtain predictions from your model and evaluate them. This will likely involve manual work to decide how the returned classes relate to the original ones.

Create a [confusion matrix](https://en.wikipedia.org/wiki/Confusion_matrix) to evaluate the classification.

### Problem 7. Grayscale (1 point)
Converting the images to grayscale should affect the classification negatively, as we lose some of the color information.

Find a way to preprocess the images to grayscale (using what you already have in Problem 4 and 5), pass them to the model, and compare the classification results to the previous ones.

### Problem 8. Deep image features (1 point)
Find a way to extract one-dimensional vectors (features) for each (non-grayscale) image, using your model. This is typically done by "short-circuiting" the model output to be an intermediate layer, while keeping the input the same. 

In case the outputs (also called feature maps) have different shapes, you can flatten them in different ways. Try to not create huge vectors; the goal is to have a relatively short sequence of numbers which describes each image.

You may find a tutorial like [this](https://towardsdatascience.com/exploring-feature-extraction-with-cnns-345125cefc9a) pretty useful but note your implementation will depend on what model (and framework) you've decided to use.

It's a good idea to save these as one or more files, so you'll spare yourself a ton of preprocessing.

### Problem 9. Putting deep image features to use (1 points)
Try to find similar images, using a similarity metric on the features you got in the previous problem. Two good metrics are `mean squared error` and `cosine similarity`. How do they work? Can you spot images that look too similar? Can you explain why?

\* If we were to take Fourier features (in a similar manner, these should be a vector of about the same length), how do they compare to the deep features; i.e., which features are better to "catch" similar images?

### * Problem 10. Explore, predict, and evaluate further
You can do a ton of things here, at your desire. For example, how does masking different areas of the image affect classification - a method known as **saliency map** ([info](https://en.wikipedia.org/wiki/Saliency_map))? Can we detect objects? Can we significantly reduce the number of features (keeping the quality) that we get? Can we reliably train a model to predict our own classes? We'll look into these in detail in the future.